# Download and Clean Dataset

In [1]:
import pandas as pd
import numpy as np

## Let's download the data and read it into a pandas dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


## Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

## Let's check the dataset for any missing values.

In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Split data into predictors and target

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

## Normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [6]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [7]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

## import the libraries

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import keras

from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


# Part A Build a baseline model

## Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [10]:
#build the regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and test the model

In [11]:
model = regression_model()

Instructions for updating:
Colocations handled automatically by placer.


## Train the model with 50 epoachs

In [12]:
# fit the model
epochs = 100
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
721/721 [==============================] - 1s 824us/step - loss: 1603.5347
Epoch 2/100
721/721 [==============================] - 0s 126us/step - loss: 1588.4648
Epoch 3/100
721/721 [==============================] - 0s 161us/step - loss: 1573.4786
Epoch 4/100
721/721 [==============================] - 0s 142us/step - loss: 1558.2746
Epoch 5/100
721/721 [==============================] - 0s 144us/step - loss: 1542.9581
Epoch 6/100
721/721 [==============================] - 0s 135us/step - loss: 1527.1333
Epoch 7/100
721/721 [==============================] - 0s 165us/step - loss: 1511.0905
Epoch 8/100
721/721 [==============================] - 0s 119us/step - loss: 1494.1659
Epoch 9/100
721/721 [==============================] - 0s 132us/step - loss: 1477.0124
Epoch 10/100
721/721 [==============================] - 0s 136us/step - loss: 1458.8929
Epoch 11/100
721/721 [==============================] - 0s 138us/step - loss: 143

## Evaluate the model using test data

In [14]:

loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val


309/309 [==============================] - 0s 112us/step


167.74194207546395

In [15]:
#  compute the mean squared error between the predicted concrete strength and the actual concrete strength
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

167.741944211972 0.0


In [16]:
#Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors
total_mse = 50
epochs = 100
mean_squared_errors = []
for i in range(0, total_mse):
    #X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mse) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 85.72862596727883
MSE 2: 60.90348075200053
MSE 3: 52.49275801328394
MSE 4: 46.76216730407912
MSE 5: 43.84716040725461
MSE 6: 42.166536448457094
MSE 7: 40.77121529687184
MSE 8: 39.66213432484846
MSE 9: 39.08243698441095
MSE 10: 38.56894680134301
MSE 11: 38.55901354570605
MSE 12: 38.689052297845244
MSE 13: 38.89058317412836
MSE 14: 38.444651156181656
MSE 15: 38.51270421346029
MSE 16: 38.645508744570044
MSE 17: 38.01266866517298
MSE 18: 37.81101424331418
MSE 19: 37.79968315420799
MSE 20: 37.26895350854374
MSE 21: 37.392743280404595
MSE 22: 37.07285324886779
MSE 23: 37.27163257413697
MSE 24: 37.17663140281505
MSE 25: 36.835530413778855
MSE 26: 37.22119810357449
MSE 27: 37.088622009869916
MSE 28: 37.247510138453016
MSE 29: 37.314227391215205
MSE 30: 37.170529461215615
MSE 31: 37.12240095663225
MSE 32: 37.30655509059869
MSE 33: 36.98660403007828
MSE 34: 36.816914524461076
MSE 35: 36.82459102247911
MSE 36: 36.90260649498998
MSE 37: 36.9545621964538
MSE 38: 37.20225523667814
MSE 39: 37.

# Compared with partB, when increasing the epochs, the mean is smaller and the standard deviation is smaller